In [40]:
import pyaudio  
import wave  

def play_audio(filename):
    #define stream chunk   
    chunk = 1024  

    #open a wav format music  
    f = wave.open(filename,"rb")  
    #instantiate PyAudio  
    p = pyaudio.PyAudio()  
    #open stream  
    stream = p.open(format = p.get_format_from_width(f.getsampwidth()),  
                    channels = f.getnchannels(),  
                    rate = f.getframerate(),  
                    output = True)  
    #read data  
    data = f.readframes(chunk)  

    #play stream  
    while data:  
        stream.write(data)  
        data = f.readframes(chunk)  

    #stop stream  
    stream.stop_stream()  
    stream.close()  

    #close PyAudio  
    p.terminate()  


In [92]:
class Exercise:
    def __init__(self, exercise_name, duration, audio=None):
        self._exercise_name = exercise_name
        self._duration = duration
        self._audio = audio
        
        self._starting = True
        
    def __str__(self):
        return self._exercise_name + ': ' + str(self._duration)
        
    def decrement(self):
        if self._starting:
            self.play_audio()
            self._starting = False
        
        self._duration -= 1
    
    def is_complete(self):
        return self._duration < 0
    
    def play_audio(self):
        if self._audio:
            play_audio(self._audio)
    
    def get_text(self):
        return self._exercise_name 
    
    def get_remaining_dur(self):
        return self._duration

In [93]:
class ExerciseController:
    def __init__(self, exercise_dict, start_audio, pause_audio, pause_duration=10):
        self._exercise_list = [Exercise(ex, duration, start_audio) for ex, duration in exercise_dict.items()]
        self._pause = None
        self._curr = None
        self._complete = False
        
        self._pause_audio = pause_audio
    
    def get_exercise_text(self):
        if self._complete:
            return None
        
        if self._curr == None:
            self._pause = True
            self._curr = Exercise('Next:\n' + self._exercise_list[0].get_text(), 10)
        
        if self._curr.is_complete():
            if self._pause:
                self._pause = False
                self._curr = self._exercise_list[0]
            else:
                self._exercise_list.remove(self._curr)
                if len(self._exercise_list) == 0:
                    self._curr = None
                    self._complete = True
                
                else:
                    self._curr = Exercise('Next:\n' + self._exercise_list[0].get_text(), 10, self._pause_audio)
                    self._pause = True
        
        ## Get the text and decrement
        text = self._curr.get_text() + '\n\n' + str(self._curr.get_remaining_dur())
        self._curr.decrement()
        return text

In [94]:
start_audio = 'audio/all-the-way.wav'
pause_audio = 'audio/yay.wav'

exercise_dict = {'Jumping Jacks': 30, 'Wall Sit': 30, 'Push Ups': 30, 'Crunches': 30, 
                 'Step-ups': 30, 'Squats': 30, 'Tricep Dips': 30, 'Plank': 30, 'High Knees': 30,
                 'Lunges': 30, 'Push-up/Rotate': 30, 'Side Plank': 30}

exercise_controller = ExerciseController(exercise_dict, start_audio, pause_audio)

In [95]:
#!/usr/bin/env python3

from tkinter import *
from tkinter import ttk
from tkinter import font
import time
    
def quit(*args):
    root.destroy()

def show_exercises():
    
    global exercise_controller
           
    exercise_text = exercise_controller.get_exercise_text()
    
    if not exercise_text:
        play_audio(pause_audio)
        txt.set('Complete!')
        return
    
    txt.set(exercise_text)
    
    root.after(1000, show_exercises)


root = Tk()
root.attributes("-fullscreen", True)
root.configure(background='black')
root.bind("<Escape>", quit)
root.bind("x", quit)
root.after(1000, show_exercises)

fnt = font.Font(family='Helvetica', size=128, weight='bold')
txt = StringVar()
txt.set('')
lbl = ttk.Label(root, textvariable=txt, font=fnt, foreground="green", background="black")
lbl.place(relx=0.5, rely=0.5, anchor=CENTER)

root.mainloop()